In [ ]:
# Create a fishnet using a raster file

from dgt.smp import nfishnet_fm_rst

imd = '/home/yuri/dgt/sample_osmimd/imd_74_v2.tif'
out = "/home/jasp/mystuff/dgt/omsvsimd74/fishnet"

fishnet = nfishnet_fm_rst(imd, 500, 500, out)

In [1]:
imd = '/home/yuri/dgt/sample_osmimd/imd_74_v2.tif'

osm_ref_tags = {
    "TABLE"     : '/home/yuri/dgt/sample_osmimd/osm_features_2021_v2.xlsx',
    "SHEET"     : 'osm_features',
    "LULC_COL"  : 'L4',
    "KEY_COL"   : "key",
    "VALUE_COL" : "value",
    "GEOM_COL"  : "geom"
}

osmdata = {
    "FILE"  : '/home/yuri/dgt/sample_osmimd/osmpt_74.xml',
    "DB"    : 'osmdb74',
    "TABLE" : "multipolygons",
    "DBSET" : "local"
}

ref_edificado = [
    '1141', '1151', '1211', '1221',
    '1222', '1223', '1231', '1241',
    '1251', '1252', '1254', '1255',
    '1256', '1257', '1253', '1612',
    '1631', '1632', '1633', '1651',
    '16', '143', '1431', '1432'
]


lulccls = 'lulc_cls'

fishnet = '/home/jasp/mystuff/dgt/osmvsimd74/fishnet/fishnet_1.shp'

#refshp = '/home/yurimtig/dgt/temp/fishnet_1.shp'

# OSM LULC result
lulcshp = '/home/yurimtig/dgt/osmlulc.shp'

# OSM/CAE intersection result
osmdissolve = '/home/yurimtig/dgt/osmdissolve.shp'

epsg = 3763

# Output
outshp = '/home/yuri/dgt/final'

In [2]:
import os
import pandas as pd
import numpy as np

from dgt.it.osm import osm_to_psql
from dgt.it.shp import dbtbl_to_shp
from dgt.wenv.grs import run_grass
from dgt.dp.torst import shp_to_rst
from dgt.rd.shp import shp_to_obj
from dgt.wt.shp import df_to_shp

from dgt.prop.sql import cols_name
from dgt.rd import tbl_to_obj
from dgt.sql.q import exec_write_q
from dgt.sql.db import create_db
from dgt.pys.oss import mkdir, fprop

In [3]:
# Prepare workspace
ws = mkdir(os.path.join(
    os.path.dirname(outshp), 'grswork'
), overwrite=True)

In [4]:
# Import data into a database
create_db(osmdata["DB"], api='psql', overwrite=True, dbset=osmdata["DBSET"])

osm_to_psql(osmdata["FILE"], osmdata["DB"])

'osmdb74'

In [5]:
osm_tags = tbl_to_obj(osm_ref_tags["TABLE"], sheet=osm_ref_tags["SHEET"])

osm_tags = osm_tags[osm_tags[osm_ref_tags["GEOM_COL"]] == 'Polygon']

In [6]:
osm_tags['sevtags'] = osm_tags[osm_ref_tags["LULC_COL"]].str.contains(';')

osm_tags = osm_tags[osm_tags.sevtags != True]

In [7]:
# Create key/value column
osm_tags.loc[:, osm_ref_tags["VALUE_COL"]] = osmdata["TABLE"] + "." + \
    osm_tags[osm_ref_tags["KEY_COL"]] + \
    "='" + osm_tags[osm_ref_tags["VALUE_COL"]] + "'"

In [8]:
# Add new column to multipolygons table
# Update it adding an LULC class

cols = cols_name(osmdata["DB"], osmdata['TABLE'], dbset=osmdata["DBSET"])

qs = [] if "lulc_cls" in cols else [(
    f"ALTER TABLE {osmdata['TABLE']} ADD COLUMN "
    "lulc_cls integer"
)]

for cls in osm_tags[osm_ref_tags["LULC_COL"]].unique():
    # Se uma feature estiver associada a duas TAGS que dizem respeito a classes
    # diferentes, a classe da feature será a última classe considerada
    # Abordagem multitag vai resolver este problema.
    __osmtags = osm_tags[osm_tags[osm_ref_tags["LULC_COL"]] == cls]
    
    qs.append((
        f"UPDATE {osmdata['TABLE']} SET lulc_cls={str(cls)} "
        f"WHERE {str(__osmtags[osm_ref_tags['VALUE_COL']].str.cat(sep=' OR '))}"
    ))

In [9]:
cols = cols_name(osmdata["DB"], osmdata['TABLE'], dbset=osmdata["DBSET"])
print(cols)

['ogc_fid', 'osm_id', 'osm_way_id', 'name', 'type', 'aeroway', 'amenity', 'admin_level', 'barrier', 'boundary', 'building', 'craft', 'geological', 'historic', 'land_area', 'landuse', 'leisure', 'man_made', 'military', '"natural"', 'office', 'place', 'shop', 'sport', 'tourism', 'waterway', 'power', 'railway', 'healthcare', 'highway', 'water', 'landcover', 'industrial', 'other_tags', 'wkb_geometry']


In [10]:
# RUN queries
exec_write_q(osmdata["DB"], qs, api='psql', dbset=osmdata["DBSET"])

In [ ]:
# Export shapefile with data
whr = " OR ".join([f"lulc_cls={c}" for c in ref_edificado])

q = (
    f'SELECT ogc_fid, osm_id, name, lulc_cls, '
    'building, amenity, landuse, '
    f'ST_Transform(wkb_geometry, {str(epsg)}) AS geom '
    f'FROM {osmdata["TABLE"]} '
    f'WHERE {whr}'
)

osmlulc = dbtbl_to_shp(
    osmdata["DB"], q, 'geom', os.path.join(ws, 'osmlulc.shp'),
    tableIsQuery=True, api='pgsql2shp', epsg=epsg,
    dbset=osmdata["DBSET"]
)

In [ ]:
# Start GRASS GIS Session
loc = 'locwork'
gb = run_grass(ws, location=loc, srs=imd)

import grass.script.setup as gsetup

gsetup.init(gb, ws, loc, 'PERMANENT')

In [ ]:
# GRASS GIS Modules
from dgt.it.shp import shp_to_grs, grs_to_shp
from dgt.it.rst import rst_to_grs
from dgt.gp.gen import dissolve
from dgt.tbl.cols import add_fields, cols_calc
from dgt.gp.ovl import grsintersection
from dgt.smp import sample_to_points

In [ ]:
# Import data
fishnetgrs = shp_to_grs(fishnet, fprop(fishnet, 'fn'))

osmlulcgrs = shp_to_grs(osmlulc, fprop(osmlulc, 'fn'), filterByReg=True)

In [ ]:
# Dissolve atraves de um atributo com valores iguais

add_fields(osmlulcgrs, {'gencol': 'integer'}, api="grass")

cols_calc(osmlulcgrs, "gencol", 1, "gencol IS NULL", ascmd=None)

osmdiss = dissolve(osmlulcgrs, 'osmdissolve', "gencol", api='grass')

In [ ]:
# Intersect fishnet feactures with osm polygons

iosm_fish = grsintersection(fishnetgrs, osmdiss, 'osm_fishnet')

In [ ]:
# Extract Raster values to points

pshp = grs_to_shp(fishnetgrs, os.path.join(ws, f'pnt_{fishnetgrs}.shp'), 'centroid')
pntgrs = shp_to_grs(pshp, fprop(pshp, 'fn'))

add_fields(pntgrs, {'imdval' : "double precision"}, api="grass")

imdgrs = rst_to_grs(imd, fprop(imd, 'fn'))

sample_to_points(pntgrs, 'imdval', imdgrs)

In [ ]:
# Export intersection result to file

ishp = grs_to_shp(iosm_fish, os.path.join(ws, iosm_fish + '.shp'), 'area')
pshp = grs_to_shp(pntgrs, os.path.join(ws, f'val_{pntgrs}.shp'), 'point')

In [ ]:
# Open Fishnet and Intersection results
fishdf = shp_to_obj(fishnet)

idf = shp_to_obj(ishp)

pdf = shp_to_obj(pshp)

In [ ]:
# Sanitize col values

idf = idf[~idf.a_cat.isna()]

idf['a_cat'] = idf.a_cat.astype(int)

# Get field with area
idf["garea"] = idf.geometry.area

In [ ]:
# Get area with OSM data in each cell
areabycell = pd.DataFrame({
    'iarea' : idf.groupby(['a_cat'])['garea'].agg('sum')
}).reset_index()

In [ ]:
# Join with original fishnet
fishdf['cellid'] = fishdf.index + 1

fishdf = fishdf.merge(areabycell, how='left', left_on='cellid', right_on='a_cat')

In [ ]:
fishdf['iarea'] = fishdf.iarea.fillna(value=0)

In [ ]:
fishdf["urbanp"] = fishdf.iarea * 100 / fishdf.geometry.area



In [ ]:
# Get IMD Values

pdf.drop([c for c in pdf.columns.values if c != 'imdval'], axis=1, inplace=True)

pdf['pid'] = pdf.index + 1

fishdf = fishdf.merge(pdf, how='left', left_on="cellid", right_on="pid")

In [ ]:
fishdf.drop(["a_cat", "pid"], axis=1, inplace=True)

In [ ]:
# Export result

df_to_shp(fishdf, outshp)

Código versão 1

In [ ]:
#Add Columns

from grass.pygrass.modules import Module
    
Module('v.db.addcolumn', intersect, columns = 'areapercent double')

In [ ]:
#Add data in the Columns

from grass.pygrass.modules import Module
    
Module('v.to.db', intersect, option = 'area', columns = 'areapercent', units = 'meters',overwrite=True)


In [ ]:
#Join tablegrs_to_shpgrass.pygrass.modules import Module

Module('v.db.join', fishnetgrs, column='cat', other_table=intersect, other_column='a_cat')




In [ ]:
#Add column in the fishnet table

Module('v.db.addcolumn', fishnetgrs, columns = 'areapercent2 double')

In [ ]:
#Update fishnet table with intersect areas values

Module('v.db.update', fishnetgrs, column='areapercent2', query_column='((round(areapercent,-2))/100)')

In [ ]:
#Drop join column

Module('v.db.dropcolumn', fishnetgrs, columns='areapercent')

In [ ]:
#Drop join column

Module('v.db.dropcolumn', fishnetgrs, columns='b_area7')

In [ ]:
# OSM/GRID intersection result to shp

fishosm = '/home/yurimtig/dgt/fishosm2.shp'
saidafish= grs_to_shp(fishnetgrs, fishosm, 'auto')

In [ ]:
#Vector grid to raster

grd_rst = Module('v.to.rast', fishnetgrs, output='gridrst2',
                 use='attr', attribute_column='areapercent2',
                overwrite=True)


In [ ]:
#Export grass to raster

from dgt.it.rst import grs_to_rst

osmvsgridrst = '/home/yurimtig/dgt/gridvsosm2.tif'

gridrst = grs_to_rst('gridrst2', osmvsgridrst)